## Tryout functions

In [ ]:
import os
import importlib
import helperfunctions

# Clear cache and reload the helperfunctions module
importlib.reload(helperfunctions)

# Import the required functions after reloading
from helperfunctions import get_database_connection, fetch_team_matches, calculate_ball_possession


In [ ]:
conn = get_database_connection()
team_name = "anderlecht"
games = fetch_team_matches(team_name, conn)

# Display the team_id of the given team_name
team_id = games.loc[games['home_team_name'].str.lower() == team_name.lower(), 'home_team_id'].iloc[0]
print(f"Team ID for {team_name}: {team_id}")
conn.close()


In [ ]:
print(games)

In [ ]:
# Re-establish the database connection
conn = get_database_connection()

# Fetch match events for the first game in the games dataframe
match_id = games.iloc[0]['match_id']
match_events = helperfunctions.fetch_match_events(match_id, conn)

# Close the connection after fetching the data
conn.close()

print(match_events.head(10))
print(match_events.describe())
print(match_events.info())

In [ ]:
import pandas as pd

# Re-establish the database connection
conn = get_database_connection()

# Fetch match events for the first game in the games dataframe
match_id = games.iloc[0]['match_id']
changes = calculate_ball_possession(match_id, conn,team_id=team_id)

# Close the connection after fetching the data
conn.close()

print(changes.head(10))


In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Filter the data to include only rows where time_difference_seconds is between -500 and 500
changes['time_difference_seconds'] = changes['time_difference'].dt.total_seconds()

# Print negative time_difference_seconds values
negative_time_diff = changes[changes['time_difference_seconds'] < 0]
print("Negative time_difference_seconds values:")
print(negative_time_diff[['time_difference_seconds']])

# Set negative time_difference_seconds to 0
changes.loc[changes['time_difference_seconds'] < 0, 'time_difference_seconds'] = 0

# Create a histogram with facets for ball_possession and a bin size of 5 seconds
sns.histplot(data=changes, x='time_difference_seconds', hue='ball_possession', multiple='dodge', bins=range(-500, 505, 5))

# Add labels and title
plt.xlabel('Time Difference (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Ball Possession Changes for Anderlecht')
plt.legend(title='Ball Possession', labels=['Opponent', 'Anderlecht'])

# Show the plot
plt.show()


In [ ]:
# Filter the data to include only rows where time_difference_seconds is between -60 and 60
filtered_changes_60 = changes[(changes['time_difference_seconds'] >= -60) & (changes['time_difference_seconds'] <= 60)]

# Create a histogram with facets for ball_possession and a bin size of 5 seconds
sns.histplot(data=filtered_changes_60, x='time_difference_seconds', hue='ball_possession', multiple='dodge', bins=range(0, 65, 5))

# Add labels and title
plt.xlabel('Time Difference (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Ball Possession Changes for Anderlecht (Zoomed In)')
plt.legend(title='Ball Possession', labels=['Opponent', 'Anderlecht'])

# Show the plot
plt.show()